In [1]:
import os
import math
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
class args(object):
    pass
    


# In[3]:

###############################################################################
# Load data
###############################################################################
fModelName = 'model1000EnTr.pth'
fullModelName = os.path.join('./',fModelName)
embedding_dim = 64
batch_size = 100
max_epoch = 500 # 500
learning_rate = 0.1
learning_rate_decay = 1
threshold = 100


class BiLingual(nn.Module):
    
    def __init__(self, vocab_size_pri,vocab_size_sec ,embedding_dim,batch_size):
        super(BiLingual, self).__init__()
        self.embeddings_pri = nn.Embedding(vocab_size_pri, embedding_dim)
        self.embeddings_sec = nn.Embedding(vocab_size_sec, embedding_dim)
        
    def cAdd(self,embeds):
        btch_len = embeds.size()[0]
        sntc_len = embeds.size()[1]
        ret = []
        for i in range(btch_len):
            splt=torch.split(embeds[i],sntc_len,1)
            tot = autograd.Variable(torch.zeros(embedding_dim),requires_grad=False).cuda()
            for j in range(sntc_len):
                tot = tot + embeds[i][j]
            ret.append(tot)
        ret=torch.stack(ret,0)
        return ret
       
    def forwardPri(self, inputs):
        embeds_pri = self.embeddings_pri(inputs)
        out_pri = self.cAdd(embeds_pri)
        return out_pri
    
    def forwardSec(self, inputs):
        embeds_sec = self.embeddings_sec(inputs)
        out_sec = self.cAdd(embeds_sec)
        return out_sec

def adjust_learning_rate(optimizer, epoch,threshold,lr_init,lr_decay_rate):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = lr_init * (lr_decay_rate ** (epoch // threshold))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

# Set the random seed manually for reproducibility.
torch.manual_seed(1111)
if torch.cuda.is_available():
    torch.cuda.manual_seed(1111)
torch.cuda.is_available(),torch.cuda.device_count()

(True, 1)

In [4]:
print('Model file %s' % fullModelName)

Model file ./model1000EnTr.pth


In [5]:
modelLoaded= torch.load(fullModelName)

In [12]:
import pickle

In [13]:
b = pickle.load( open( "./model.pck", "rb" ) )

In [17]:
b.embeddings_pri.weight

Parameter containing:
 4.2834e-02  4.1015e-01  2.7584e-01  ...  -2.1113e-01  1.1106e-01  4.8122e-01
 5.7446e-01 -2.2783e+00 -4.8139e-02  ...  -5.8571e-01  8.9599e-01 -4.8478e-01
 1.3280e+00 -8.1286e-03 -1.1540e+00  ...   6.8914e-01 -1.1108e+00  1.0914e+00
                ...                   ⋱                   ...                
 1.3663e+00 -8.7230e-01  8.5720e-01  ...   2.4645e-01  5.6051e-02 -1.7939e+00
 4.0651e-01 -7.3624e-01  1.1642e+00  ...  -7.0021e-01 -3.9584e-01  8.1520e-01
-6.2293e-01  1.0769e+00 -1.4651e+00  ...   9.6729e-01 -2.2503e+00 -1.4580e+00
[torch.cuda.FloatTensor of size 3060x64 (GPU 0)]